<a href="https://colab.research.google.com/github/ahelmasri87/-AAI614_elmasri/blob/main/data_visualization_using_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install dash dash-bootstrap-components pandas plotly

In [ ]:
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd


In [ ]:
# Load the dataset
data = pd.read_excel('/content/hos110-test.xlsx')

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [ ]:
app.layout = html.Div([
    html.H1('Providers Claims Analysis Dashboard'),
    dcc.Dropdown(
        id='provider-dropdown',
        options=[{'label': i, 'value': i} for i in data['Provider_name'].unique()],
        value=data['Provider_name'].unique()[0],
        clearable=False
    ),
    dcc.Graph(id='claims-time-series'),
    dcc.Graph(id='net-vs-dedc-scatter'),
    dcc.Graph(id='benefit-type-distribution-providers')
])


In [ ]:
@app.callback(
    Output('claims-time-series', 'figure'),
    Input('provider-dropdown', 'value'))
def update_time_series(selected_provider):
    # Filter data for the selected provider
    filtered_data = data[data['Provider_name'] == selected_provider]

    # Aggregate data by date
    aggregated_data = filtered_data.groupby('InvoiceDate').agg({'total_claim1':'sum'}).reset_index()

    # Apply a rolling average for smoothing (optional)
    aggregated_data['smoothed_claims'] = aggregated_data['total_claim1'].rolling(window=7, center=True).mean()

    # Create the figure using the aggregated and optionally smoothed data
    fig = px.line(aggregated_data, x='InvoiceDate', y='smoothed_claims',
                  title='Total Claims Over Time - Smoothed',
                  labels={'smoothed_claims': 'Total Claims ($)', 'InvoiceDate': 'Date'},
                  markers=True)  # Add markers for individual data points

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Total Claims ($)",
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )

    return fig

In [ ]:
# Callback to update the scatter plot based on selected provider
@app.callback(
    Output('net-vs-dedc-scatter', 'figure'),
    Input('provider-dropdown', 'value'))
def update_scatter_plot(selected_provider):
    filtered_data = data[data['Provider_name'] == selected_provider]
    fig = px.scatter(filtered_data, x='net_app', y='dedc_app', color='insured_class',
                     title='Net Amount vs. Deductible by Insured Class')
    return fig

In [ ]:
@app.callback(
    Output('benefit-type-distribution-providers', 'figure'),
    Input('provider-dropdown', 'value'))
def update_bar_chart(selected_provider):
    # Filter data for the selected provider
    filtered_data = data[data['Provider_name'] == selected_provider]

    # Group and aggregate data
    aggregated_data = filtered_data.groupby(['admission_type', 'benfit_type']).size().reset_index(name='counts')

    # Create a bar chart
    fig = px.bar(aggregated_data, x='admission_type', y='counts', color='benfit_type',
                 title='Distribution of Benefit Types by Admission Type',
                 labels={'counts': 'Count of Claims', 'admission_type': 'Admission Type', 'benfit_type': 'benfit_type'})

    return fig



In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>